In [ ]:
## Top 4 U.S. DEA controlled substances
##   codein, fentanyl, morphine, and oxycodone

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## store the linux environment variable WORKSPACE_CDE into a python variable DATASET
##  Curated Data Repository (CDR)
DATASET = %env WORKSPACE_CDR

In [ ]:
## find the concept_id of the 'fentanyl' in 'Ingredient' class of 'RxNorm' vocabulary concept_id = 1154029
input_sql = f'''
SELECT c.concept_id, c.concept_name, c.vocabulary_id, c.concept_class_id
FROM fc-aou-cdr-prod-ct.C2024Q3R5.concept c
WHERE lower(c.concept_name) in ('codeine','dizzepam', 'fentanyl', 'morphine', 'oxycodone')
  AND c.vocabulary_id = 'RxNorm' AND c.concept_class_id = 'Ingredient'
'''

df_ingridient = pd.read_gbq(input_sql)

In [ ]:
## Find all drug exposure events in all drugs whose ingredient is in
## ('codeine', 'fentanyl', 'morphine', 'oxycodone') with concept_id's
## (1201620, 1154029, 1110410, 1124957) 
input_sql = f'''
SELECT de.person_id, 
  ca.ancestor_concept_id AS drug_ingridient_concept_id,
  de.drug_concept_id, 
  de.drug_exposure_start_datetime, 
  de.drug_exposure_end_datetime,
  de.days_supply,
  de.quantity
FROM `{DATASET}.concept_ancestor` ca
 JOIN `{DATASET}.concept` c
   ON ca.descendant_concept_id = c.concept_id
 JOIN `{DATASET}.drug_exposure` de
   ON de.drug_concept_id = c.concept_id
WHERE ca.ancestor_concept_id in (1201620, 1154029, 1110410, 1124957)               
  AND c.domain_id = 'Drug'
  AND c.invalid_reason IS NULL
'''

df_events = pd.read_gbq(input_sql)

In [ ]:
## find the participant enrollment date (=primary consent date)
input_sql = f'''
SELECT DISTINCT person_id, MIN(observation_date) AS primary_consent_date
    FROM `{DATASET}.concept`
     JOIN `{DATASET}.concept_ancestor` on concept_id = ancestor_concept_id
     JOIN `{DATASET}.observation` on descendant_concept_id = observation_source_concept_id
    WHERE concept_name = 'Consent PII'
      AND concept_class_id = 'Module'
    GROUP BY 1
'''
df_consent = pd.read_gbq(input_sql)

In [ ]:
## retrieve age at consent and demography
input_sql = f'''
SELECT p.person_id, p.sex_at_birth, p.race, p.age_at_consent,
   p.state_of_residence
  FROM `{DATASET}.cb_search_person` p
'''
df_search_person = pd.read_gbq(input_sql)

In [ ]:
## join primary consent date with demography
df_person = pd.merge(df_search_person, df_consent, on='person_id', how='inner')

In [ ]:
## join drug events and person
df_join = pd.merge(df_events, df_person, on='person_id', how='inner')
[df_join['person_id'].nunique(), df_join.shape[0]] # count person, count events

In [ ]:
## sort 
df_join.sort_values(by=['person_id', 'drug_exposure_start_datetime', 'drug_exposure_end_datetime'],
                             ascending=[True, True, True], inplace=True)

In [ ]:
## count tne missing values in column 'drug_exposure_end_datetime' and total row count
[df_join['drug_exposure_end_datetime'].isnull().sum(), df_join.shape[0]]

In [ ]:
## create a drug episode dataframe
# Ensure datetime dtype and sort properly
df = df_join.sort_values(['person_id', 'drug_exposure_start_datetime']).copy()

# Gap between consecutive events per person (in days)
g = df.groupby('person_id')
df['gap_days'] = g['drug_exposure_start_datetime'].diff().dt.days

# Start a new episode when the gap is >= 30 days (first event per person gets NaN -> treat as no new episode)
df['new_episode'] = (df['gap_days'].fillna(0) >= 30).astype(int)

# Cumulative sum within each person gives a episode index starting at 0
df['drug_episode_id'] = g['new_episode'].transform('cumsum')

# a 1-based label
df['drug_episode_order'] = df['drug_episode_id'] + 1

# Summaries per episode
df_episode = (
    df.groupby(['person_id', 'drug_episode_id'])
      .agg(
          drug_episode_start_datetime=('drug_exposure_start_datetime', 'min'),
          drug_episode_end_datetime=('drug_exposure_start_datetime', 'max'),
          n_events=('drug_exposure_start_datetime', 'size')
      )
      .assign(drug_episode_span_days=lambda x: (x['drug_episode_end_datetime'] - x['drug_episode_start_datetime']).dt.days)
      .reset_index()
)

In [ ]:
df_episode['drug_episode_span_days'].describe()

In [ ]:
## dipslay the number of episodes and unique patients with span days > 1 year
[ df_episode.loc[df_episode['drug_episode_span_days'] > (365*1)].shape[0],
  df_episode.loc[df_episode['drug_episode_span_days'] > (365*1)]['person_id'].nunique() ]

In [ ]:
## dipslay the number of episodes and unique patients with span days > 2 years
[ df_episode.loc[df_episode['drug_episode_span_days'] > (365*2)].shape[0],
  df_episode.loc[df_episode['drug_episode_span_days'] > (365*2)]['person_id'].nunique() ]

In [ ]:
df_join['person_id'].nunique()

In [ ]:
df_episode.head()

In [ ]:
df_join['drug_concept_id'].nunique()